In [1]:
!pip install --no-index -f /kaggle/input/download-ultralytics /kaggle/input/download-ultralytics/ultralytics-8.1.0-py3-none-any.whl

Looking in links: /kaggle/input/download-ultralytics
Processing /kaggle/input/download-ultralytics/ultralytics-8.1.0-py3-none-any.whl
Processing /kaggle/input/download-ultralytics/thop-0.1.1.post2209072238-py3-none-any.whl (from ultralytics==8.1.0)
Processing /kaggle/input/download-ultralytics/hub_sdk-0.0.3-py3-none-any.whl (from ultralytics==8.1.0)


In [2]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ultralytics.engine.results import Results

In [3]:
!mkdir trained_model
!cp /kaggle/input/yolo-train-kidney-1-dense-val-kidney-3-dense-1303/hacking_human_vasculature/train_kidney_1_dense_val_kidney_3_dense_size_1303/weights/best.pt /kaggle/working/trained_model

In [4]:
def add_masks(masks):
    result = 255*(np.sum(masks, axis=0))
    result = result.clip(0, 255).astype("uint8")
    return result

In [5]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

In [6]:
def extract_id_from_result(result: Results):
    dataset_name = result.path.split('/')[-3]
    file_name = result.path.split('/')[-1].split('.')[0]
    return f'{dataset_name}_{file_name}'

In [7]:
def get_rle_from_result(result: Results):
    if not result.masks:
        return '1 0'
    else:
        masks_array = result.masks.data.cpu().numpy()
        combined_mask = add_masks(masks_array)
        return rle_encode(combined_mask)

In [8]:
import cv2
imgsz = cv2.imread('/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0000.tif').shape[:2]
# imgsz = cv2.imread('/kaggle/input/blood-vessel-segmentation/train/kidney_3_sparse/images/0000.tif').shape[:2]

In [9]:
model = YOLO('/kaggle/working/trained_model/best.pt')

In [10]:
# source = '/kaggle/input/blood-vessel-segmentation/train/kidney_3_sparse/images/102*.tif'
source = '/kaggle/input/blood-vessel-segmentation/test/**/*.tif'
results = model.predict(source, 
                        stream=True, 
                        device=[0,1], 
                        retina_masks=True, 
                        imgsz=imgsz,
                        conf=0.1)

In [11]:
submission_list = []
for result in results:
    img_id = extract_id_from_result(result)
    rle = get_rle_from_result(result)
    submission_list.append({
        'id': img_id,
        'rle': rle
    })

df = pd.DataFrame(submission_list, columns=['id', 'rle'])

df.to_csv('submission.csv', index=False)



WARNING ⚠️ imgsz=[1303, 912] must be multiple of max stride 32, updating to [1312, 928]
image 1/6 /kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0000.tif: 1312x928 (no detections), 21.2ms
image 2/6 /kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0001.tif: 1312x928 (no detections), 21.2ms
image 3/6 /kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0002.tif: 1312x928 (no detections), 21.2ms
image 4/6 /kaggle/input/blood-vessel-segmentation/test/kidney_6/images/0000.tif: 1312x928 (no detections), 21.2ms
image 5/6 /kaggle/input/blood-vessel-segmentation/test/kidney_6/images/0001.tif: 1312x928 (no detections), 21.2ms
image 6/6 /kaggle/input/blood-vessel-segmentation/test/kidney_6/images/0002.tif: 1312x928 (no detections), 21.2ms
Speed: 9.1ms preprocess, 21.2ms inference, 12.6ms postprocess per image at shape (1, 3, 1312, 928)
